## This notebook contains codes i used to answer the graded quiz questions.
TIME : 90mins <br>
Student Name: Omeh Chukwuemeka <br>
Student_ID: c01e2e

### Dataset Description
The dataset for the remainder of this quiz is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.

### Attribute Information:

Date, time year-month-day hour:minute:second

Appliances, energy use in Wh

lights, energy use of light fixtures in the house in Wh

T1, Temperature in kitchen area, in Celsius

RH_1, Humidity in kitchen area, in %

T2, Temperature in living room area, in Celsius

RH_2, Humidity in living room area, in %

T3, Temperature in laundry room area

RH_3, Humidity in laundry room area, in %

T4, Temperature in office room, in Celsius

RH_4, Humidity in office room, in %

T5, Temperature in bathroom, in Celsius

RH_5, Humidity in bathroom, in %

T6, Temperature outside the building (north side), in Celsius

RH_6, Humidity outside the building (north side), in %

T7, Temperature in ironing room , in Celsius

RH_7, Humidity in ironing room, in %

T8, Temperature in teenager room 2, in Celsius

RH_8, Humidity in teenager room 2, in %

T9, Temperature in parents room, in Celsius

RH_9, Humidity in parents room, in %

To, Temperature outside (from Chievres weather station), in Celsius

Pressure (from Chievres weather station), in mm Hg

RH_out, Humidity outside (from Chievres weather station), in %

Wind speed (from Chievres weather station), in m/s

Visibility (from Chievres weather station), in km

Tdewpoint (from Chievres weather station), Â °C

rv1, Random variable 1, nondimensional

rv2, Random variable 2, nondimensional

<b>To answer some questions, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a  random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set.</b>

In [1]:
#importing necessary libraries
import pandas as pd             #For data analysis
import numpy as np             #For numeric computations
import matplotlib.pyplot as plt  #For data visualization
import seaborn as sns             #For data visualization
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### LOADING DATASET

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
#let drop the lights and date columns as stated in the instruction
df_drop = df.drop(columns = ['date','lights'], axis = 1)
df_drop.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


#### NORMALIZING OUR DATASET

In [4]:
#importing necessary libaries and normalization proper
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
scaler = MinMaxScaler()

normalised_df = pd.DataFrame(scaler.fit_transform(df_drop), columns=df_drop.columns)
features_df = normalised_df.drop(columns=['Appliances'])
target = normalised_df['Appliances']


#Now, we split our dataset into the training and testing dataset.
from sklearn.model_selection import train_test_split
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(features_df, target, test_size=0.3, random_state=42)

linear_model_norm = LinearRegression()
#fit the model to the training dataset
linear_model_norm.fit(X_train_norm, y_train_norm)
#obtain predictions
predicted_values = linear_model_norm.predict(X_test_norm)

### MODEL EVALUATION AND ANSWERING QUIZ QUESTIONS

### Question 1
In linear regression, L2 regularization is equivalent to imposing a:

In [5]:
#ANSWER
#GUASSIAN PRIOR

### Question 2
Cross validation:

In [6]:
#MY ANSWER
#IS GUARANTEED TO PREVENT OVERFITTING

### Question 3
Ridge regression:

In [7]:
#MY ANS
#Is more sensitive to outliers than ordinary least square.

### Question 4
In the different terms of the bias-variance tradeoff, which of the following is substantially more harmful to the test error than the training error?

In [8]:
#MY ANS
#VARIANCE

### Question 5
What can you use to find the best fit line for data in Linear Regression

In [9]:
#MY ANS
#Least Square Error

### Question 6
Which of the following sentence is true about outliers in Linear Regression:

In [10]:
#My Ans
#Linear regression is sensitive to outliers

### Question 7
How many coefficients do you need to estimate a simple linear regression model (One independent variable)?

In [11]:
#My Ans
# 2

### Question 8
Adding more basis functions in a linear model:

In [12]:
#MY ANS
#Decrease model bias

### Question 9
A best fit line relating X and Y has a R-Squared value of 0.75. How do I interpret this information?

In [13]:
#My Ans
#The model is performing with accuracy of 75%

### Question 10
The Lasso can be interpreted as least-squares linear regression where:

In [14]:
#MY ANS
#Weights are regularized with the L1 Norm

### Question 11
Which of these is not an assumption of Linear Regression?

In [15]:
#MY ANS
#Independence

### Question 12
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

### NORMALIZING ORIGINAL DATASET TO ANSWER QUESTION 12

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
LR_12 = LinearRegression()

#let create a copy of original dataset
df_12 = df_drop.copy()
scaled_12 = MinMaxScaler()

normalised_12 = pd.DataFrame(scaled_12.fit_transform(df_12), columns=df_12.columns)
features_12 = normalised_12[['T2']]
target_12 = normalised_12[['T6']]
X_train_12, X_test_12,y_train_12, y_test_12 = train_test_split(features_12, target_12, test_size = 0.3, random_state = 42)

LR_12.fit(X_train_12, y_train_12)
predicted_12 = LR_12.predict(X_test_12)

In [17]:
#Evaluating metrics for question 12
#ANSWER = 0.64 though i failed it in quiz
R2_SCORE_12 = r2_score(y_test_12, predicted_12)
print('The R-SQUARED  for question 12 regression model in 2 DP is:', round(R2_SCORE_12, 2))

The R-SQUARED  for question 12 regression model in 2 DP is: 0.64


### Question 13
What is the Mean Absolute Error (in two decimal places)?

In [18]:
#ANSWER. THE MEAN ABSOLUTE ERROR FOR OUR MODEL 
#MAE = 
from sklearn.metrics import mean_absolute_error, mean_squared_error
MAE = mean_absolute_error(y_test_norm, predicted_values)
print('The mean absolute error from our original model in 2DP is', round(MAE, 2))

The mean absolute error from our original model in 2DP is 0.05


### Question 14
What is the Residual Sum of Squares (in two decimal places)?

In [19]:
#ANSWER
#rss = 
#let check the Residual sum of squares erro,RSS
rss = np.sum(np.square(y_test_norm - predicted_values))
print('The residual sum of squares  from our original model in 2DP is', round(rss, 2))

The residual sum of squares  from our original model in 2DP is 45.35


### Question 15
What is the Root Mean Squared Error (in three decimal places)?

In [20]:
#ANSWER
#RMSE = 
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test_norm,predicted_values))
print('The RMSE value is of our original model in 3 DP is:', round(rmse, 3))

The RMSE value is of our original model in 3 DP is: 0.088


### Question 16
What is the Coefficient of Determination (in two decimal places)? 

In [21]:
#ANSWER
#let calculate the coefficient of determination
from sklearn.metrics import r2_score
r_squared = r2_score(y_test_norm, predicted_values)
print('The coefficient of determination also called r_squared in 2 DP  is:', round(r_squared, 2))

The coefficient of determination also called r_squared in 2 DP  is: 0.15


### Question 17
Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [22]:
#ANSWER. From the table below RH_2 have the lowest weight and RH_1 has the highest weight.
#comparing the effects of regularisation and their weights
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model_norm, X_train_norm, 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


### Question 18
Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [23]:
#Ridge regression
#ANSWER: From the result, their is NO change in value of Root mean squared error as it still same with normal linear regression
#model. Is still same 0.088
from sklearn.linear_model import Ridge, Lasso, ElasticNet
ridge_reg = Ridge(alpha = 0.4)
ridge_reg.fit(X_train_norm, y_train_norm)
ridge_predict = ridge_reg.predict(X_test_norm)
#metrics

RMSE_RIDGE = np.sqrt(mean_squared_error(y_test_norm, ridge_predict))
print('The RMSE  for redge regression is:', round(RMSE_RIDGE, 3))

The RMSE  for redge regression is: 0.088


### Question 19
Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [24]:
#LASSO REGRESSION. From the table, we have 4 features that have non-zero feature weight.
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(X_train_norm, y_train_norm)
lasso_predict = lasso_reg.predict(X_test_norm)

def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

lasso_weights_df = get_weights_df(lasso_reg, X_train_norm, 'Lasso_weight')
lasso_weights_df

,Features,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


### Question 20
What is the new RMSE with the Lasso Regression (in 3 decimal places)? 

In [25]:
#The RMSE is = 0.094
RMSE_LASSO = np.sqrt(mean_squared_error(y_test_norm, lasso_predict))
print('The RMSE  for Lasso regression is:', round(RMSE_LASSO, 3))

The RMSE  for Lasso regression is: 0.094


### PLEASE NOTE
Though, i got some questions wrong during the quiz involing the model, but i decided to do make some corrections in this submission code.